# Feature Engineering

## Importación de librerías

In [3]:
#!pip install geopandas

In [4]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import re
from shapely.geometry import Point
import geopandas as gpd

In [5]:
data = pd.read_csv("./resultados/cabaventa_preproc.csv", index_col = 0)

## Creación de nuevas features a partir de la descripción de las propiedades

Se crea una columna con el título y la descripción de las propiedades para la extracción de features adicionales

In [6]:
data['texto'] = data['title_cleaned'] + " " + data['description_cleaned']

Detección de propiedades con cochera. Incluye una expresión regular para excluir a aquellos registros en la venta no incluye la cochera.

In [7]:
pattern_cochera = r"(?<!s\/)(?<!sin )(\d|con|la|las|c\/)?(cochera\.?)(?!s?\s?desde)(?!s?\s?\w*?\s?(op|cort|\d))( fija)?"

x = data['texto'].str.extract(pattern_cochera)

x.bfill(axis=1, inplace=True)
x = x.iloc[:,0]
x.loc[~x.isnull()] = True  # not nan
x.loc[x.isnull()] = False   # nan

data['has_cochera'] = x

Se crea una función de detección de amenities y características que pueden ser relevantes para calcular el precio de una propiedad en el texto libre de la publicación.

In [8]:
def apply_regex(columna, value):
  pattern_sin = r"((sin|no tiene|no contiene|no hay|no es|no esta|no incluye|no se permite|no se permiten|poca|nula)\s+("+ value +"))"
  pattern_ok = r"("+ value +")"
  # return Falso cuando no hay ninguno de los lugares
  if re.search(pattern_sin, columna, re.M|re.I) is not None:
    return False
  elif re.search(pattern_ok, columna, re.M|re.I) is not None:
    return True
  else:
    return False

In [9]:
amenities = ['patio', 'jardin', 'balcon', 'terraza', 'parrilla', 'sum', 'pileta|piscina', 'luminoso|luminosidad|mucha luz', 'laundry|lavadero|lavarropas', 'baulera', 'gimnasio|gym', 'seguridad|vigilancia', 'vestidor', 'a estrenar', 
             'pool|ping pong|metegol|microcine', 'portero|porteria|encargado', 'jacuzzi|sauna|solarium|yacuzzi', 'apto profesional', 'amenities', 'por escalera', 'a reciclar|para reciclar', 'categoria|primera linea', 'reciclado|reciclada',
             'en pozo|entrega en|de pozo|emprendimiento', 'tiro balanceado|radiante'] 

for i in amenities:
  col_name = "has_"+ i.replace(' ', '_').split("|")[0]
  data[col_name] = data['texto'].apply(lambda x: apply_regex(str(x), i))

In [10]:
filter_col = [col for col in data if col.startswith('has')]

for col in filter_col:
  print(col, data[col].mean())

has_cochera 0.32175959675907606
has_patio 0.1455708066901335
has_jardin 0.08034617066922163
has_balcon 0.5755764304013663
has_terraza 0.2853096165462082
has_parrilla 0.2533898481597967
has_sum 0.2403303409635292
has_pileta 0.22596904875965926
has_luminoso 0.4263397971298244
has_laundry 0.5307637832996605
has_baulera 0.17667826123180105
has_gimnasio 0.11658786528087313
has_seguridad 0.2143987836120889
has_vestidor 0.11819166441023932
has_a_estrenar 0.09920643185936556
has_pool 0.018537418508258525
has_portero 0.095259419716315
has_jacuzzi 0.18538459936264606
has_apto_profesional 0.1254295890525088
has_amenities 0.16568078148757576
has_por_escalera 0.04857220220366166
has_a_reciclar 0.014017620961862907
has_categoria 0.144966778446606
has_reciclado 0.1005186311470288
has_en_pozo 0.05301909978963154
has_tiro_balanceado 0.12807481618795694


## Feature: Cálculo de distancia al subte

Se carga un csv con las coordenadas geográficas de las estaciones de subte de CABA

In [11]:
url='https://drive.google.com/uc?id=1oxCEj_enxBVjBsDuvuy5w4HF02HasCWl'
subtes = pd.read_csv(url)
subtes.sample()

,long,lat,id,estacion,linea
45,-58.381535,-34.617937,46.0,INDEPENDENCIA,E


Se crea una función para crear un punto con las coordenadas geográficas

In [12]:
def from_x_y(df, x, y):
    gdf = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [13]:
caba_geo = from_x_y(data, 'lon', 'lat')
subtes_geo = from_x_y(subtes, 'long', 'lat')

C:\Users\Usuario\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Usuario\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Se realiza una proyección para poder calcular las distancias

In [14]:
caba_gkba = caba_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
subtes_gkba = subtes_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")

Se define una función que se queda con la distancia, el nombre y la línea de la estación más cercana

In [15]:
def distancia_subte(x):
  distancias = []
  for y in range(len(subtes_gkba)):
    est = subtes_gkba.geometry[y]
    dist = x.distance(est)
    distancias.append(dist)
  estacion = np.argmin(distancias)
  return min(distancias), subtes_gkba.iloc[estacion]['estacion'], subtes_gkba.iloc[estacion]['linea']

In [16]:
caba_gkba['dist_subte'], caba_gkba['estacion_subte_cercana'], caba_gkba['linea_subte_cercana'] = zip(*caba_gkba['geometry'].apply(lambda x: distancia_subte(x)))

In [17]:
data = data.join(caba_gkba[['dist_subte', 'estacion_subte_cercana', 'linea_subte_cercana']])

## Export

In [18]:
data.to_csv('resultados/cabaventa_feature.csv')

In [19]:
import datetime
end_time = time.time()
elapsed = str(datetime.timedelta(seconds=end_time - start_time))
print(f"Time execution is {elapsed}")

Time execution is 0:10:06.874014
